In [24]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import f1_score, classification_report
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from transformers import AdamW
from tqdm import tqdm

In [25]:
language="tr"
modelname="TurkishBert"
input = "Dodiom"
test = "Dodiom"

In [26]:
if input == "gpt4":
  gpt4_flag=True
else:
  gpt4_flag=False

In [27]:
# JP encoding='euc-jp' other encoding normal CHECK ALWAYS DATASETS
df_train = pd.read_csv(f'dodiom_tr_train.csv')
df_eval = pd.read_csv(f'dodiom_tr_eval.csv')
df_test = pd.read_csv(f'dodiom_tr_test.csv')

In [28]:
def get_category_mapping(language):
    if language in ['en', 'jp']:
        category_mapping = {
            'figurative': 'idiom',
            'literal': 'nonidiom'
        }
    elif language == 'tr':
        category_mapping = {
            'mecaz': 'idiom',
            'gerçek': 'nonidiom'
        }
    elif language == 'it':
        category_mapping = {
            'figurato': 'idiom',
            'letterale': 'nonidiom'
        }
    else:
        raise ValueError(f"Language '{language}' is not supported.")

    return category_mapping

In [29]:
category_mapping = get_category_mapping(language)
print(category_mapping)

{'mecaz': 'idiom', 'gerçek': 'nonidiom'}


In [30]:
if gpt4_flag or language in ['en', 'jp']:
  df_train['category'] = df_train['category'].map(category_mapping)
df_train

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words
0,adım atmak,Yeni kariyerine ilk adımını attı.,idiom,1.000000,4,0,0,"[3, 4]","['adımını', 'attı']","['yen', 'kariyer', 'ilk', 'adım', 'atmak', '.']","['Yeni', 'kariyerine', 'ilk', 'adımını', 'attı..."
1,adım atmak,Sanat yaşamına adım attığı ilk yıllardan bu ya...,idiom,0.750000,3,0,1,"[2, 3]","['adım', 'attığı']","['sanat', 'Yaşam', 'adım', 'atmak', 'ilk', 'yı...","['Sanat', 'yaşamına', 'adım', 'attığı', 'ilk',..."
2,adım atmak,Bu hayata adım atmak kolay mı sanıyorsun?,idiom,0.666667,2,0,1,"[2, 3]","['adım', 'atmak']","['bu', 'hayat', 'adım', 'atmak', 'kolay', 'mı'...","['Bu', 'hayata', 'adım', 'atmak', 'kolay', 'mı..."
3,adım atmak,Veri bilimine ilk defa adım attılar.,idiom,1.000000,3,0,0,"[4, 5]","['adım', 'attılar']","['veri', 'bilim', 'ilk', 'defa', 'adım', 'atma...","['Veri', 'bilimine', 'ilk', 'defa', 'adım', 'a..."
4,adım atmak,"Çiftin yeni bir hayata adım attığı bu gecede, ...",idiom,0.750000,3,0,1,"[4, 5]","['adım', 'attığı']","['çift', 'yen', 'bir', 'hayat', 'adım', 'atmak...","['Çiftin', 'yeni', 'bir', 'hayata', 'adım', 'a..."
...,...,...,...,...,...,...,...,...,...,...,...
4047,üzerinde durmak,Apple tamamen su üzerinde duran ilk mağazasını...,nonidiom,1.000000,4,0,0,"[3, 4]","['üzerinde', 'duran']","['Apple', 'tamamen', 'su', 'Üzer', 'durmak', '...","['Apple', 'tamamen', 'su', 'üzerinde', 'duran'..."
4048,üzerinde durmak,Üzerinde durdukları buz parçasının kırılmaya b...,nonidiom,1.000000,5,0,0,"[0, 1]","['Üzerinde', 'durdukları']","['Üzer', 'durmak', 'buz', 'parça', 'kırmak', '...","['Üzerinde', 'durdukları', 'buz', 'parçasının'..."
4049,üzerinde durmak,Sen hiç mi tek ayak üzerinde durmadın,nonidiom,1.000000,3,0,0,"[5, 6]","['üzerinde', 'durmadın']","['sen', 'hiç', 'Mi', 'tek', 'ayak', 'Üzer', 'd...","['Sen', 'hiç', 'mi', 'tek', 'ayak', 'üzerinde'..."
4050,üzerinde durmak,"Yatağın, kapının, sandalyenin ve hatta halının...",nonidiom,1.000000,4,0,0,"[8, 11]","['üzerinde', 'duruyordu']","['yatak', ',', 'kapı', ',', 'sandalye', 've', ...","['Yatağın', ',', 'kapının', ',', 'sandalyenin'..."


In [31]:
if gpt4_flag or language in ['en', 'jp']:
  df_eval['category'] = df_eval['category'].map(category_mapping)
df_eval

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words
0,adım atmak,Projede kritik adımlar atılmadan önce durum de...,idiom,1.000000,4,0,0,"[2, 3]","['adımlar', 'atılmadan']","['proje', 'kritik', 'adım', 'atmak', 'önce', '...","['Projede', 'kritik', 'adımlar', 'atılmadan', ..."
1,adım atmak,Barışmak üzere bir adım attı.,idiom,0.750000,3,0,1,"[3, 4]","['adım', 'attı']","['barışmak', 'üzere', 'bir', 'adım', 'atmak', ...","['Barışmak', 'üzere', 'bir', 'adım', 'attı', '.']"
2,adım atmak,AÖF bütünleme sınavında geri adım mı atılacak?,idiom,0.666667,2,0,1,"[4, 6]","['adım', 'atılacak']","['Aöf', 'bütünlemek', 'sınav', 'geri', 'adım',...","['AÖF', 'bütünleme', 'sınavında', 'geri', 'adı..."
3,adım atmak,Bugün o kadar çok adım attım ki aşırı yoruldum.,nonidiom,0.750000,3,0,1,"[4, 5]","['adım', 'attım']","['bugün', 'o', 'kadar', 'çok', 'adım', 'atmak'...","['Bugün', 'o', 'kadar', 'çok', 'adım', 'attım'..."
4,adım atmak,Adımını atarken dikkat et.,nonidiom,0.750000,3,0,1,"[0, 1]","['Adımını', 'atarken']","['adım', 'atmak', 'dikkat', 'etmek', '.']","['Adımını', 'atarken', 'dikkat', 'et', '.']"
...,...,...,...,...,...,...,...,...,...,...,...
720,üzerinde durmak,Sence hoca derste bu konunun üzerinde daha çok...,nonidiom,0.600000,3,2,0,"[5, 8]","['üzerinde', 'durur']","['se', 'Ho', 'ders', 'bu', 'konu', 'Üzer', 'da...","['Sence', 'hoca', 'derste', 'bu', 'konunun', '..."
721,üzerinde durmak,Üzerinde durduğu buz katmanının kırılmasıyla s...,nonidiom,1.000000,4,0,0,"[0, 1]","['Üzerinde', 'durduğu']","['Üzer', 'durmak', 'buz', 'katman', 'kırmak', ...","['Üzerinde', 'durduğu', 'buz', 'katmanının', '..."
722,üzerinde durmak,Üzerinde durdukları kolonu yıkmaya çalıştı.,nonidiom,1.000000,5,0,0,"[0, 1]","['Üzerinde', 'durdukları']","['Üzer', 'durmak', 'Kolo', 'yıkmak', 'çalışmak...","['Üzerinde', 'durdukları', 'kolonu', 'yıkmaya'..."
723,üzerinde durmak,Ben öyle bir ipin üzerinde asla duramam,nonidiom,1.000000,3,0,0,"[4, 6]","['üzerinde', 'duramam']","['ben', 'Ö', 'bir', 'ip', 'Üzer', 'asıl', 'dur...","['Ben', 'öyle', 'bir', 'ipin', 'üzerinde', 'as..."


In [32]:
if language in ['en', 'jp']:
  df_test['category'] = df_test['category'].map(category_mapping)

In [33]:
df_test

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words
0,adım atmak,"Sen bu adama adım mı attın ki, o sana yakınlaş...",idiom,0.75,3,0,1,"[3, 5]","['adım', 'attın']","['se', 'bu', 'ada', 'adım', 'mı', 'atmak', 'ki...","['Sen', 'bu', 'adama', 'adım', 'mı', 'attın', ..."
1,adım atmak,Bütün zorluklara rağmen yeni bir işe adım attı.,idiom,1.00,3,0,0,"[6, 7]","['adım', 'attı']","['bütün', 'zor', 'rağmen', 'yen', 'bir', 'işem...","['Bütün', 'zorluklara', 'rağmen', 'yeni', 'bir..."
2,adım atmak,Adım attığı bu hayatın başına ne gibi olaylar ...,idiom,0.75,3,0,1,"[0, 1]","['Adım', 'attığı']","['adım', 'atmak', 'bu', 'hayat', 'baş', 'ne', ...","['Adım', 'attığı', 'bu', 'hayatın', 'başına', ..."
3,adım atmak,"Bölüm başkanı, akademik dünyaya adım atan genç...",idiom,0.50,1,0,1,"[5, 6]","['adım', 'atan']","['bölüm', 'başka', ',', 'akademik', 'Dünya', '...","['Bölüm', 'başkanı', ',', 'akademik', 'dünyaya..."
4,adım atmak,Yeni bir deneyime adım atıyorum.,idiom,0.75,3,0,1,"[3, 4]","['adım', 'atıyorum']","['yen', 'bir', 'deneyim', 'adım', 'atmak', '.']","['Yeni', 'bir', 'deneyime', 'adım', 'atıyorum'..."
...,...,...,...,...,...,...,...,...,...,...,...
2079,üzerinde durmak,Bu ipin üzerinde duramıyorum.,nonidiom,1.00,5,0,0,"[2, 3]","['üzerinde', 'duramıyorum']","['bu', 'ip', 'Üzer', 'durmak', '.']","['Bu', 'ipin', 'üzerinde', 'duramıyorum', '.']"
2080,üzerinde durmak,İpin üzerinde tek ayağıyla duruyordu.,nonidiom,1.00,5,0,0,"[1, 4]","['üzerinde', 'duruyordu']","['ip', 'Üzer', 'tek', 'Ayağ', 'durmak', '.']","['İpin', 'üzerinde', 'tek', 'ayağıyla', 'duruy..."
2081,üzerinde durmak,Bilgisayarın üzerinde kitapları duruyor,nonidiom,1.00,4,0,0,"[1, 3]","['üzerinde', 'duruyor']","['bilgisayar', 'Üzer', 'kitap', 'durmak']","['Bilgisayarın', 'üzerinde', 'kitapları', 'dur..."
2082,üzerinde durmak,Üzerinde çilekler duran tartı bana uzattı.,nonidiom,1.00,1,0,0,"[0, 2]","['Üzerinde', 'duran']","['Üzer', 'Çilek', 'durmak', 'tartı', 'ban', 'u...","['Üzerinde', 'çilekler', 'duran', 'tartı', 'ba..."


In [34]:
df_train['category'] = df_train['category'].str.lower()
df_eval['category'] = df_eval['category'].str.lower()
df_test['category'] = df_test['category'].str.lower()

In [35]:
df_train.head()

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words
0,adım atmak,Yeni kariyerine ilk adımını attı.,idiom,1.000000,4,0,0,"[3, 4]","['adımını', 'attı']","['yen', 'kariyer', 'ilk', 'adım', 'atmak', '.']","['Yeni', 'kariyerine', 'ilk', 'adımını', 'attı..."
1,adım atmak,Sanat yaşamına adım attığı ilk yıllardan bu ya...,idiom,0.750000,3,0,1,"[2, 3]","['adım', 'attığı']","['sanat', 'Yaşam', 'adım', 'atmak', 'ilk', 'yı...","['Sanat', 'yaşamına', 'adım', 'attığı', 'ilk',..."
2,adım atmak,Bu hayata adım atmak kolay mı sanıyorsun?,idiom,0.666667,2,0,1,"[2, 3]","['adım', 'atmak']","['bu', 'hayat', 'adım', 'atmak', 'kolay', 'mı'...","['Bu', 'hayata', 'adım', 'atmak', 'kolay', 'mı..."
3,adım atmak,Veri bilimine ilk defa adım attılar.,idiom,1.000000,3,0,0,"[4, 5]","['adım', 'attılar']","['veri', 'bilim', 'ilk', 'defa', 'adım', 'atma...","['Veri', 'bilimine', 'ilk', 'defa', 'adım', 'a..."
4,adım atmak,"Çiftin yeni bir hayata adım attığı bu gecede, ...",idiom,0.750000,3,0,1,"[4, 5]","['adım', 'attığı']","['çift', 'yen', 'bir', 'hayat', 'adım', 'atmak...","['Çiftin', 'yeni', 'bir', 'hayata', 'adım', 'a..."


In [36]:
df_eval.head()

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words
0,adım atmak,Projede kritik adımlar atılmadan önce durum de...,idiom,1.000000,4,0,0,"[2, 3]","['adımlar', 'atılmadan']","['proje', 'kritik', 'adım', 'atmak', 'önce', '...","['Projede', 'kritik', 'adımlar', 'atılmadan', ..."
1,adım atmak,Barışmak üzere bir adım attı.,idiom,0.750000,3,0,1,"[3, 4]","['adım', 'attı']","['barışmak', 'üzere', 'bir', 'adım', 'atmak', ...","['Barışmak', 'üzere', 'bir', 'adım', 'attı', '.']"
2,adım atmak,AÖF bütünleme sınavında geri adım mı atılacak?,idiom,0.666667,2,0,1,"[4, 6]","['adım', 'atılacak']","['Aöf', 'bütünlemek', 'sınav', 'geri', 'adım',...","['AÖF', 'bütünleme', 'sınavında', 'geri', 'adı..."
3,adım atmak,Bugün o kadar çok adım attım ki aşırı yoruldum.,nonidiom,0.750000,3,0,1,"[4, 5]","['adım', 'attım']","['bugün', 'o', 'kadar', 'çok', 'adım', 'atmak'...","['Bugün', 'o', 'kadar', 'çok', 'adım', 'attım'..."
4,adım atmak,Adımını atarken dikkat et.,nonidiom,0.750000,3,0,1,"[0, 1]","['Adımını', 'atarken']","['adım', 'atmak', 'dikkat', 'etmek', '.']","['Adımını', 'atarken', 'dikkat', 'et', '.']"


In [37]:
df_test.head()

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words
0,adım atmak,"Sen bu adama adım mı attın ki, o sana yakınlaş...",idiom,0.75,3,0,1,"[3, 5]","['adım', 'attın']","['se', 'bu', 'ada', 'adım', 'mı', 'atmak', 'ki...","['Sen', 'bu', 'adama', 'adım', 'mı', 'attın', ..."
1,adım atmak,Bütün zorluklara rağmen yeni bir işe adım attı.,idiom,1.00,3,0,0,"[6, 7]","['adım', 'attı']","['bütün', 'zor', 'rağmen', 'yen', 'bir', 'işem...","['Bütün', 'zorluklara', 'rağmen', 'yeni', 'bir..."
2,adım atmak,Adım attığı bu hayatın başına ne gibi olaylar ...,idiom,0.75,3,0,1,"[0, 1]","['Adım', 'attığı']","['adım', 'atmak', 'bu', 'hayat', 'baş', 'ne', ...","['Adım', 'attığı', 'bu', 'hayatın', 'başına', ..."
3,adım atmak,"Bölüm başkanı, akademik dünyaya adım atan genç...",idiom,0.50,1,0,1,"[5, 6]","['adım', 'atan']","['bölüm', 'başka', ',', 'akademik', 'Dünya', '...","['Bölüm', 'başkanı', ',', 'akademik', 'dünyaya..."
4,adım atmak,Yeni bir deneyime adım atıyorum.,idiom,0.75,3,0,1,"[3, 4]","['adım', 'atıyorum']","['yen', 'bir', 'deneyim', 'adım', 'atmak', '.']","['Yeni', 'bir', 'deneyime', 'adım', 'atıyorum'..."


In [38]:
# Encode the 'category' column
label_encoder = LabelEncoder()
df_train['category_encoded'] = label_encoder.fit_transform(df_train['category'])
df_eval['category_encoded'] = label_encoder.transform(df_eval['category'])
df_test['category_encoded'] = label_encoder.transform(df_test['category'])

In [39]:
df_train

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words,category_encoded
0,adım atmak,Yeni kariyerine ilk adımını attı.,idiom,1.000000,4,0,0,"[3, 4]","['adımını', 'attı']","['yen', 'kariyer', 'ilk', 'adım', 'atmak', '.']","['Yeni', 'kariyerine', 'ilk', 'adımını', 'attı...",0
1,adım atmak,Sanat yaşamına adım attığı ilk yıllardan bu ya...,idiom,0.750000,3,0,1,"[2, 3]","['adım', 'attığı']","['sanat', 'Yaşam', 'adım', 'atmak', 'ilk', 'yı...","['Sanat', 'yaşamına', 'adım', 'attığı', 'ilk',...",0
2,adım atmak,Bu hayata adım atmak kolay mı sanıyorsun?,idiom,0.666667,2,0,1,"[2, 3]","['adım', 'atmak']","['bu', 'hayat', 'adım', 'atmak', 'kolay', 'mı'...","['Bu', 'hayata', 'adım', 'atmak', 'kolay', 'mı...",0
3,adım atmak,Veri bilimine ilk defa adım attılar.,idiom,1.000000,3,0,0,"[4, 5]","['adım', 'attılar']","['veri', 'bilim', 'ilk', 'defa', 'adım', 'atma...","['Veri', 'bilimine', 'ilk', 'defa', 'adım', 'a...",0
4,adım atmak,"Çiftin yeni bir hayata adım attığı bu gecede, ...",idiom,0.750000,3,0,1,"[4, 5]","['adım', 'attığı']","['çift', 'yen', 'bir', 'hayat', 'adım', 'atmak...","['Çiftin', 'yeni', 'bir', 'hayata', 'adım', 'a...",0
...,...,...,...,...,...,...,...,...,...,...,...,...
4047,üzerinde durmak,Apple tamamen su üzerinde duran ilk mağazasını...,nonidiom,1.000000,4,0,0,"[3, 4]","['üzerinde', 'duran']","['Apple', 'tamamen', 'su', 'Üzer', 'durmak', '...","['Apple', 'tamamen', 'su', 'üzerinde', 'duran'...",1
4048,üzerinde durmak,Üzerinde durdukları buz parçasının kırılmaya b...,nonidiom,1.000000,5,0,0,"[0, 1]","['Üzerinde', 'durdukları']","['Üzer', 'durmak', 'buz', 'parça', 'kırmak', '...","['Üzerinde', 'durdukları', 'buz', 'parçasının'...",1
4049,üzerinde durmak,Sen hiç mi tek ayak üzerinde durmadın,nonidiom,1.000000,3,0,0,"[5, 6]","['üzerinde', 'durmadın']","['sen', 'hiç', 'Mi', 'tek', 'ayak', 'Üzer', 'd...","['Sen', 'hiç', 'mi', 'tek', 'ayak', 'üzerinde'...",1
4050,üzerinde durmak,"Yatağın, kapının, sandalyenin ve hatta halının...",nonidiom,1.000000,4,0,0,"[8, 11]","['üzerinde', 'duruyordu']","['yatak', ',', 'kapı', ',', 'sandalye', 've', ...","['Yatağın', ',', 'kapının', ',', 'sandalyenin'...",1


In [40]:
df_eval

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words,category_encoded
0,adım atmak,Projede kritik adımlar atılmadan önce durum de...,idiom,1.000000,4,0,0,"[2, 3]","['adımlar', 'atılmadan']","['proje', 'kritik', 'adım', 'atmak', 'önce', '...","['Projede', 'kritik', 'adımlar', 'atılmadan', ...",0
1,adım atmak,Barışmak üzere bir adım attı.,idiom,0.750000,3,0,1,"[3, 4]","['adım', 'attı']","['barışmak', 'üzere', 'bir', 'adım', 'atmak', ...","['Barışmak', 'üzere', 'bir', 'adım', 'attı', '.']",0
2,adım atmak,AÖF bütünleme sınavında geri adım mı atılacak?,idiom,0.666667,2,0,1,"[4, 6]","['adım', 'atılacak']","['Aöf', 'bütünlemek', 'sınav', 'geri', 'adım',...","['AÖF', 'bütünleme', 'sınavında', 'geri', 'adı...",0
3,adım atmak,Bugün o kadar çok adım attım ki aşırı yoruldum.,nonidiom,0.750000,3,0,1,"[4, 5]","['adım', 'attım']","['bugün', 'o', 'kadar', 'çok', 'adım', 'atmak'...","['Bugün', 'o', 'kadar', 'çok', 'adım', 'attım'...",1
4,adım atmak,Adımını atarken dikkat et.,nonidiom,0.750000,3,0,1,"[0, 1]","['Adımını', 'atarken']","['adım', 'atmak', 'dikkat', 'etmek', '.']","['Adımını', 'atarken', 'dikkat', 'et', '.']",1
...,...,...,...,...,...,...,...,...,...,...,...,...
720,üzerinde durmak,Sence hoca derste bu konunun üzerinde daha çok...,nonidiom,0.600000,3,2,0,"[5, 8]","['üzerinde', 'durur']","['se', 'Ho', 'ders', 'bu', 'konu', 'Üzer', 'da...","['Sence', 'hoca', 'derste', 'bu', 'konunun', '...",1
721,üzerinde durmak,Üzerinde durduğu buz katmanının kırılmasıyla s...,nonidiom,1.000000,4,0,0,"[0, 1]","['Üzerinde', 'durduğu']","['Üzer', 'durmak', 'buz', 'katman', 'kırmak', ...","['Üzerinde', 'durduğu', 'buz', 'katmanının', '...",1
722,üzerinde durmak,Üzerinde durdukları kolonu yıkmaya çalıştı.,nonidiom,1.000000,5,0,0,"[0, 1]","['Üzerinde', 'durdukları']","['Üzer', 'durmak', 'Kolo', 'yıkmak', 'çalışmak...","['Üzerinde', 'durdukları', 'kolonu', 'yıkmaya'...",1
723,üzerinde durmak,Ben öyle bir ipin üzerinde asla duramam,nonidiom,1.000000,3,0,0,"[4, 6]","['üzerinde', 'duramam']","['ben', 'Ö', 'bir', 'ip', 'Üzer', 'asıl', 'dur...","['Ben', 'öyle', 'bir', 'ipin', 'üzerinde', 'as...",1


In [41]:
df_test

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words,category_encoded
0,adım atmak,"Sen bu adama adım mı attın ki, o sana yakınlaş...",idiom,0.75,3,0,1,"[3, 5]","['adım', 'attın']","['se', 'bu', 'ada', 'adım', 'mı', 'atmak', 'ki...","['Sen', 'bu', 'adama', 'adım', 'mı', 'attın', ...",0
1,adım atmak,Bütün zorluklara rağmen yeni bir işe adım attı.,idiom,1.00,3,0,0,"[6, 7]","['adım', 'attı']","['bütün', 'zor', 'rağmen', 'yen', 'bir', 'işem...","['Bütün', 'zorluklara', 'rağmen', 'yeni', 'bir...",0
2,adım atmak,Adım attığı bu hayatın başına ne gibi olaylar ...,idiom,0.75,3,0,1,"[0, 1]","['Adım', 'attığı']","['adım', 'atmak', 'bu', 'hayat', 'baş', 'ne', ...","['Adım', 'attığı', 'bu', 'hayatın', 'başına', ...",0
3,adım atmak,"Bölüm başkanı, akademik dünyaya adım atan genç...",idiom,0.50,1,0,1,"[5, 6]","['adım', 'atan']","['bölüm', 'başka', ',', 'akademik', 'Dünya', '...","['Bölüm', 'başkanı', ',', 'akademik', 'dünyaya...",0
4,adım atmak,Yeni bir deneyime adım atıyorum.,idiom,0.75,3,0,1,"[3, 4]","['adım', 'atıyorum']","['yen', 'bir', 'deneyim', 'adım', 'atmak', '.']","['Yeni', 'bir', 'deneyime', 'adım', 'atıyorum'...",0
...,...,...,...,...,...,...,...,...,...,...,...,...
2079,üzerinde durmak,Bu ipin üzerinde duramıyorum.,nonidiom,1.00,5,0,0,"[2, 3]","['üzerinde', 'duramıyorum']","['bu', 'ip', 'Üzer', 'durmak', '.']","['Bu', 'ipin', 'üzerinde', 'duramıyorum', '.']",1
2080,üzerinde durmak,İpin üzerinde tek ayağıyla duruyordu.,nonidiom,1.00,5,0,0,"[1, 4]","['üzerinde', 'duruyordu']","['ip', 'Üzer', 'tek', 'Ayağ', 'durmak', '.']","['İpin', 'üzerinde', 'tek', 'ayağıyla', 'duruy...",1
2081,üzerinde durmak,Bilgisayarın üzerinde kitapları duruyor,nonidiom,1.00,4,0,0,"[1, 3]","['üzerinde', 'duruyor']","['bilgisayar', 'Üzer', 'kitap', 'durmak']","['Bilgisayarın', 'üzerinde', 'kitapları', 'dur...",1
2082,üzerinde durmak,Üzerinde çilekler duran tartı bana uzattı.,nonidiom,1.00,1,0,0,"[0, 2]","['Üzerinde', 'duran']","['Üzer', 'Çilek', 'durmak', 'tartı', 'ban', 'u...","['Üzerinde', 'çilekler', 'duran', 'tartı', 'ba...",1


In [42]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [43]:
# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-uncased")

# Tokenize the data
train_encodings = tokenizer(list(df_train['submission']), truncation=True, padding=True, return_tensors="pt")
val_encodings = tokenizer(list(df_eval['submission']), truncation=True, padding=True, return_tensors="pt")
test_encodings = tokenizer(list(df_test['submission']), truncation=True, padding=True, return_tensors="pt")
# Convert to torch tensors
train_labels = torch.tensor(df_train['category_encoded'].values)
val_labels = torch.tensor(df_eval['category_encoded'].values)
test_labels = torch.tensor(df_test['category_encoded'].values)

# Prepare to datasets
train_dataset = TextDataset(train_encodings, train_labels)
val_dataset = TextDataset(val_encodings, val_labels)
test_dataset = TextDataset(test_encodings, test_labels)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
num_labels = len(label_encoder.classes_)
model = AutoModelForSequenceClassification.from_pretrained("dbmdz/bert-base-turkish-uncased", num_labels=num_labels)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-6)

epochs = 4
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    # Average loss across all batches
    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs} | Train Loss: {avg_train_loss}")

    # Evaluation step
    model.eval()
    total_eval_accuracy = 0
    for batch in tqdm(val_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        accuracy = (predictions == batch['labels']).cpu().numpy().mean() * 100
        total_eval_accuracy += accuracy

    avg_val_accuracy = total_eval_accuracy / len(val_loader)
    print(f"Epoch {epoch + 1}/{epochs} | Validation Accuracy: {avg_val_accuracy:.2f}%")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/507 [00:00<?, ?it/s]<ipython-input-42-845b27163e62>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-42-845b27163e62>:8: U

Epoch 1/4 | Train Loss: 0.48247641845215


100%|██████████| 91/91 [00:01<00:00, 49.80it/s]


Epoch 1/4 | Validation Accuracy: 83.43%


100%|██████████| 507/507 [01:01<00:00,  8.20it/s]


...

100%|██████████| 91/91 [00:01<00:00, 49.84it/s]


Epoch 3/4 | Validation Accuracy: 87.69%


100%|██████████| 507/507 [01:01<00:00,  8.21it/s]


Epoch 4/4 | Train Loss: 0.1165973438231065


 16%|█▋        | 15/91 [00:00<00:01, 49.04it/s]

In [ ]:
# Ensure the model is in evaluation mode
model.eval()

# Move the model to the appropriate device
model.to(device)

# Initialize variables to track accuracy
total_correct = 0
total_predictions = 0

# Store true labels and predictions for F1 score calculation
all_labels = []
all_predictions = []

# No gradient updates needed for evaluation
with torch.no_grad():
    for batch in test_loader:
        # Move batch to the same device as the model
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)

        # Get predictions
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        # Update tracking variables
        total_correct += (predictions == batch['labels']).sum().item()
        total_predictions += predictions.size(0)

        # Store predictions and true labels
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(batch['labels'].cpu().numpy())

# Calculate accuracy
test_accuracy = total_correct / total_predictions
print(f'Test Accuracy: {test_accuracy:.4f}')

# Calculate F1 scores
f1_weighted = f1_score(all_labels, all_predictions, average='weighted')
f1_macro = f1_score(all_labels, all_predictions, average='macro')

print(f'Weighted F1 Score: {f1_weighted:.4f}')
print(f'Macro F1 Score: {f1_macro:.4f}')


In [ ]:
# Generate classification report
report = classification_report(all_labels, all_predictions, digits=4)
print(f'Classification Report for {modelname}, language {language.upper()}, Train {input.upper()}, Test {test.upper()}:')
print(report)